# Boston Housing Dataset

## Step 0: Basic library imports

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import sklearn
import pandas as pd
import seaborn as sns

## Step 1: Load dataset

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
boston.keys()

In [ ]:
# to see the shape of dataset
type(boston.data), boston.data.shape

In [ ]:
print(boston.DESCR)

### Pandas dataframe

In [ ]:
# Now let’s convert it into pandas Dataframe
df_feat = pd.DataFrame(boston.data)
print(df_feat.head())

In [ ]:
boston.feature_names

In [ ]:
# convert the index to the column names
df_feat.columns = boston.feature_names
df_feat.head()

In [ ]:
df_tar = pd.DataFrame(boston.target)
df_tar.columns = ["MEDV"]

In [ ]:
df = pd.concat([df_feat, df_tar], axis=1)
df.head()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

## Step 2: Data Visualisation and Feature selection

### Correlation

In [ ]:
ax, fig = plt.subplots(figsize=(10,8))
sns.heatmap(df.corr().sort_values(by=['MEDV']), annot=True, cmap='coolwarm')
plt.show()

- From correlation matrix, we see TAX and RAD are highly correlated features. 
- The columns LSTAT,  PTRAIO, INDUS, TAX, NOX, RM has a correlation score above 0.5 with MEDV which is a good indication of using as predictors.

Lets drop "RAD" as a feature

In [ ]:
del df["RAD"]

### Outliers

In [ ]:
from scipy import stats

fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.boxplot(y=k, data=df, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

Columns like CRIM, ZN, RM, B seems to have outliers. Let's see the outliers percentage in every column.

In [ ]:
for k, v in df.items(): #k=column_name, v=values
    q1 = v.quantile(0.25)
    q3 = v.quantile(0.75)
    irq = q3 - q1
    v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
    perc = np.shape(v_col)[0] * 100.0 / np.shape(df)[0]
    print("Column %s outliers = %.2f%%" % (k, perc))

Let's try and remove outliers form the target variable

In [ ]:
df["MEDV"].describe()

In [ ]:
df["MEDV"].quantile(0.25) - 1.5*(df["MEDV"].quantile(0.75) - df["MEDV"].quantile(0.25))

In [ ]:
df["MEDV"].quantile(0.75) + 1.5*(df["MEDV"].quantile(0.75) - df["MEDV"].quantile(0.25))

In [ ]:
df["MEDV"].value_counts().sort_index()

In [ ]:
df = df[~(df['MEDV'] == 50.0)]
df.shape

### Missing values

In [ ]:
df.isnull().sum()

### Data distribution

In [ ]:
fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.distplot(v, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

- The histogram shows that columns CRIM, ZN, B has highly skewed distributions. 
- Also MEDV looks to have a normal distribution (the predictions) and other colums seem to have norma or bimodel ditribution of data except CHAS (which is a discrete variable).

Let's try to remove the skewness of the data trough log transformation.

In [ ]:
for col in ["CRIM", "ZN", "B"]:
    print(col)
    print(df[col].skew())
    df[col] = np.log1p(df[col]) 

## Step 3: Defining Training and Test Set

In [ ]:
X, y = df.loc[:,df.columns!="MEDV"].values, df["MEDV"].values

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Step 4: Data Scaling

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler().fit(X_train)
X_train = std.transform(X_train)
X_test = std.transform(X_test)

In [ ]:
pd.DataFrame(X_train, columns=boston.feature_names).describe().round(3)

## Step 5: Modelling

### 5.1. Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, y_train)

y_pred = lm.predict(X_test)

The linear regression reports 95% confidence intervals for the regression parameters, and we can visualize what this means using the seaborn library, which plots the regression line and highlights the 95% (by default) confidence interval for the regression line:

In [ ]:
import seaborn as sns
ax = sns.regplot(y_test, y_pred);
ax.set_xlabel("Prices: $y_i$")
ax.set_ylabel("Predicted prices: $\hat{y}_i$")
ax.set_title("Prices vs Predicted prices: $y_i$ vs $\hat{y}$")

In [ ]:
#calculating the error manually
np.sum((y_pred - y_test)**2)/y_test.shape[0]

In [ ]:
#or you can import the Sklearn mse 
mse_ols = sklearn.metrics.mean_squared_error(y_test, y_pred)
print(mse_ols)

- For quantifying the model performace we will be calculating the coefficient of determination, R2.

- The coefficient of determination for a model is a useful statistic in regression analysis, as it often describes how "good" that model is at making predictions.

- The values for R2 range from `0 to 1`, which captures the percentage of `squared correlation between the predicted and actual values of the target variable`. 

- A model with an R2 of 0 is no better than a model that always predicts the mean of the target variable, whereas a model with an R2 of 1 perfectly predicts the target variable. 

- Any value between 0 and 1 indicates what percentage of the target variable, using this model, can be explained by the features. A model can be given a negative R2 as well, which indicates that the model is arbitrarily worse than one that always predicts the mean of the target variable.

- For the performance_metric function in the code cell below, you will need to implement the following:
    - Use r2_score from sklearn.metrics to perform a performance calculation between y_true and y_predict.
    - Assign the performance score to the score variable.

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

### 5.2. Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

In [ ]:
ax = sns.regplot(y_test, y_pred);
ax.set_xlabel("Prices: $y_i$")
ax.set_ylabel("Predicted prices: $\hat{y}_i$")
ax.set_title("Prices vs Predicted prices: $y_i$ vs $\hat{y}$")

In [ ]:
mse_lasso = sklearn.metrics.mean_squared_error(y_test, y_pred)
print(mse_lasso)

### 5.3. Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_test)

In [ ]:
ax = sns.regplot(y_test, y_pred);
ax.set_xlabel("Prices: $y_i$")
ax.set_ylabel("Predicted prices: $\hat{y}_i$")
ax.set_title("Prices vs Predicted prices: $y_i$ vs $\hat{y}$")

In [ ]:
mse_ridge = sklearn.metrics.mean_squared_error(y_test, y_pred)
print(mse_ridge)

## Summary

In [ ]:
mse_all = {'ols'    : mse_ols,
           'lasso'  : mse_lasso, 
           'ridge'  : mse_ridge}

for name, mse, in mse_all.items():
    print(f"{name:<7}: {mse:.4}")

In [ ]:
def reg_equation(coefs, names=None):
    "Create the regression equation with coefficents and names"
    if names is None:
        # Assign numbers to betas
        names = [f"X{_}" for _ in range(len(coefs))]
    equation = " \t+ \n".join("{:>6.3f}*{}".format(c, n.lower())
                                   for c, n in zip(coefs, names))
    return equation

In [ ]:
print("OLS")
print(reg_equation(lm.coef_, names=boston.feature_names))
print()

print("Lasso")
lasso = Lasso()
lasso.fit(X_train, y_train)
print(reg_equation(lasso.coef_, names=boston.feature_names))
print()

print("Ridge")
ridge = Ridge()
ridge.fit(X_train, y_train)
print(reg_equation(ridge.coef_, names=boston.feature_names))
print()

We can try some non prametric regression techniques: 
- SVR with kernal rbf, 
- DecisionTreeRegressor, 
- KNeighborsRegressor 